# BabyView vs. THINGs: Quantifying Change Within Semantic Categories

**Main question:** Do categories (words) within the same CDI semantic category have **consistent movements**—i.e. do their BV→THINGs arrows look similar in both **direction** and **magnitude** (distance) within each semantic group?

**Movement is a vector:** each arrow has direction and magnitude. We quantify consistency of both within each CDI semantic category.

**Goals:**
1. **Magnitude (distance)** — For each word we compute the displacement length (BV→THINGs). We summarize by semantic category (mean, std) and measure **magnitude consistency** within each category (coefficient of variation; low CV = similar distances within the category).
2. **Direction** — We treat each arrow as a unit direction. Within each semantic category we compute **direction alignment** (mean resultant length). High alignment = arrows point in similar directions within that category.

**Input:** Coordinate CSVs produced by `visualize_tsne_umap_bv_things.py` (e.g. `umap_bv_things_coordinates.csv`, `tsne_bv_things_coordinates.csv`) in `bv_things_results_clip/` or `bv_things_results_dinov3/`.

## 1. Imports and paths

We use `pandas` for tables, `numpy` for numeric operations, and `matplotlib` for plotting. The notebook assumes it is run from the `manuscript-2026` directory (or you set `RESULTS_BASE` to point to the folder that contains `bv_things_results_clip` and `bv_things_results_dinov3`).

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Base directory containing bv_things_results_clip/ and bv_things_results_dinov3/.
# Run the notebook from manuscript-2026/ so that Path(".") finds these folders.
RESULTS_BASE = Path(".").resolve()
if not (RESULTS_BASE / "bv_things_results_clip").exists():
    # If run from project root, try manuscript-2026
    candidate = RESULTS_BASE / "object-detection" / "analysis" / "manuscript-2026"
    if candidate.exists():
        RESULTS_BASE = candidate
print("Results base:", RESULTS_BASE)
print("CLIP UMAP exists:", (RESULTS_BASE / "bv_things_results_clip" / "umap_bv_things_coordinates.csv").exists())

Results base: /home/j7yang/babyview-projects/vss2026/object-detection/analysis/manuscript-2026/object-detection/analysis/manuscript-2026
CLIP UMAP exists: False


## 2. Load coordinate CSV and detect axis columns

Each CSV has columns like `category`, `cdi_category`, and four coordinate columns: `bv_*_x`, `bv_*_y`, `things_*_x`, `things_*_y` (where `*` is `umap` or `tsne`). We detect these from the column names so the same code works for both UMAP and t-SNE.

In [12]:
def load_bv_things_coordinates(csv_path):
    """Load a *_{tsne,umap}_bv_things_coordinates.csv and return (df, bv_x, bv_y, things_x, things_y)."""
    df = pd.read_csv(csv_path)
    # Find coordinate columns: bv_*_x, bv_*_y, things_*_x, things_*_y
    coord_cols = [c for c in df.columns if c not in ("category", "cdi_category")]
    bv_x = [c for c in coord_cols if c.startswith("bv_") and c.endswith("_x")][0]
    bv_y = [c for c in coord_cols if c.startswith("bv_") and c.endswith("_y")][0]
    things_x = [c for c in coord_cols if c.startswith("things_") and c.endswith("_x")][0]
    things_y = [c for c in coord_cols if c.startswith("things_") and c.endswith("_y")][0]
    return df, bv_x, bv_y, things_x, things_y

# Example: load CLIP UMAP results (change path or loop over multiple files as needed)
csv_path = RESULTS_BASE / "bv_things_results_clip" / "umap_bv_things_coordinates.csv"
df, bv_x, bv_y, things_x, things_y = load_bv_things_coordinates(csv_path)
print("Loaded:", csv_path.name)
print("Coordinate columns: bv:", bv_x, bv_y, "| things:", things_x, things_y)
df.head(10)

FileNotFoundError: [Errno 2] No such file or directory: '/home/j7yang/babyview-projects/vss2026/object-detection/analysis/manuscript-2026/object-detection/analysis/manuscript-2026/bv_things_results_clip/umap_bv_things_coordinates.csv'

## 3. Compute displacement magnitude and direction (unit vector)

For each row (word):
- **Displacement:** `(dx, dy) = (things_x - bv_x, things_y - bv_y)`. **Magnitude** = Euclidean length.
- **Direction:** Unit vector `(ux, uy) = (dx, dy) / magnitude` (for nonzero displacement). We use this to measure how aligned arrows are within each semantic category.

In [13]:
dx = (df[things_x] - df[bv_x]).values
dy = (df[things_y] - df[bv_y]).values
magnitude = np.sqrt(dx * dx + dy * dy)
df = df.copy()
df["displacement"] = magnitude

# Unit direction (avoid division by zero; zero-displacement rows get NaN and are dropped later for direction stats)
eps = 1e-12
length = np.maximum(magnitude, eps)
ux = dx / length
uy = dy / length
df["dir_x"] = ux
df["dir_y"] = uy

print("Displacement (magnitude) summary (all words):")
print(df["displacement"].describe())
df[["category", "cdi_category", "displacement", "dir_x", "dir_y"]].head(10)

## 4. Summarize by semantic category (CDI category)

**Movement is a vector:** each BV→THINGs arrow has **direction** and **magnitude** (distance). We assess consistency within each semantic category on both.

**Displacement by category:** For each `cdi_category` we compute mean and std of `displacement` (and count of words).

**Direction alignment by category:** For each `cdi_category`, we take the unit direction vectors `(dir_x, dir_y)`, compute their **mean vector**, then the **length** of that mean vector (mean resultant length):
- **≈ 1**: arrows in that category point in almost the same direction (high direction consistency).
- **≈ 0**: arrows are scattered (no consistent direction).

**Magnitude consistency by category:** We use the **coefficient of variation** (CV = std/mean of displacement within the category). Lower CV means displacements are more similar in size within that category (consistent magnitude). We also store **magnitude_consistency** = 1/(1+CV) so that higher = more consistent (on a 0–1 scale).

In [14]:
def direction_alignment(ux, uy):
    """Mean resultant length: length of the mean of unit vectors (ux, uy). Returns 0 if empty."""
    if len(ux) == 0:
        return np.nan
    mx, my = np.mean(ux), np.mean(uy)
    return np.sqrt(mx * mx + my * my)

# Group by semantic category
grp = df.groupby("cdi_category", sort=True)
displacement_stats = grp["displacement"].agg(["mean", "std", "count", "median"]).round(4)
displacement_stats.columns = ["displacement_mean", "displacement_std", "n_words", "displacement_median"]

# Direction alignment per category
alignment = grp.apply(lambda g: direction_alignment(g["dir_x"].values, g["dir_y"].values), include_groups=False)
alignment.name = "direction_alignment"
displacement_stats["direction_alignment"] = alignment.round(4)
# Magnitude consistency: CV = std/mean within category (lower = more consistent). magnitude_consistency = 1/(1+cv) so higher = more consistent.
displacement_stats["displacement_cv"] = (displacement_stats["displacement_std"] / displacement_stats["displacement_mean"]).round(4)
displacement_stats["magnitude_consistency"] = (1 / (1 + displacement_stats["displacement_cv"])).round(4)

# Sort by mean displacement (optional: sort by alignment to see most consistent categories first)
displacement_stats = displacement_stats.sort_values("displacement_mean", ascending=False)
print("Per semantic category: displacement (mean, std, n), direction alignment (0–1), and magnitude consistency (0–1):")
displacement_stats

## 5. Plot: Mean displacement by semantic category

Bar plot of mean BV→THINGs displacement per CDI category. Error bars = standard deviation. This shows *how much* BV and THINGs differ on average within each semantic group.

In [15]:
fig, ax = plt.subplots(figsize=(10, 5))
cats = displacement_stats.index
x = np.arange(len(cats))
ax.bar(x, displacement_stats["displacement_mean"], yerr=displacement_stats["displacement_std"].fillna(0),
       capsize=3, color="steelblue", alpha=0.8, edgecolor="navy")
ax.set_xticks(x)
ax.set_xticklabels(cats, rotation=45, ha="right")
ax.set_ylabel("Mean displacement (BV → THINGs)")
ax.set_title("BV vs THINGs: Mean displacement by semantic category (CDI)")
ax.set_xlabel("Semantic category")
plt.tight_layout()
plt.show()

## 6. Plot: Direction alignment by semantic category

Bar plot of *direction alignment* (mean resultant length) per CDI category. Values close to **1** mean arrows in that category point in very similar directions ("BV and THINGs differ in similar ways" within that category). Values close to **0** mean directions are scattered.

In [16]:
fig, ax = plt.subplots(figsize=(10, 5))
x = np.arange(len(cats))
ax.bar(x, displacement_stats["direction_alignment"], color="coral", alpha=0.8, edgecolor="darkred")
ax.set_xticks(x)
ax.set_xticklabels(cats, rotation=45, ha="right")
ax.set_ylabel("Direction alignment (0 = scattered, 1 = same direction)")
ax.set_title("BV vs THINGs: Direction consistency within semantic category")
ax.set_xlabel("Semantic category")
ax.set_ylim(0, 1.05)
plt.tight_layout()
plt.show()

## 6a. Plot: Magnitude consistency by semantic category

Bar plot of **magnitude consistency** (1/(1+CV)) per CDI category. Higher = displacements within that category are more similar in size (consistent magnitude). This is the "distance" part of the movement vector.

In [17]:
fig, ax = plt.subplots(figsize=(10, 5))
cats = displacement_stats.index
x = np.arange(len(cats))
ax.bar(x, displacement_stats["magnitude_consistency"], color="seagreen", alpha=0.8, edgecolor="darkgreen")
ax.set_xticks(x)
ax.set_xticklabels(cats, rotation=45, ha="right")
ax.set_ylabel("Magnitude consistency (0 = mixed sizes, 1 = similar distances)")
ax.set_title("BV vs THINGs: Magnitude consistency within semantic category")
ax.set_xlabel("Semantic category")
ax.set_ylim(0, 1.05)
plt.tight_layout()
plt.show()

## 6b. Answer: Do categories within the same CDI semantic category have consistent movements?

**Movement is a vector:** each BV→THINGs arrow has **direction** and **magnitude** (distance). So "consistent movement" can mean both:
1. **Consistent direction** — arrows point the same way within the category.
2. **Consistent magnitude** — arrows have similar lengths within the category.

**Metrics:**
- **Direction alignment** (mean resultant length): high (e.g. > 0.7) = similar direction; low (e.g. < 0.4) = scattered directions.
- **Magnitude consistency** (1/(1+CV) of displacement within category): high (e.g. > 0.7) = similar distances; low = mixed sizes.

Below we summarize both across semantic categories and list which have high vs low consistency on direction and magnitude.

In [18]:
# Summarize consistency of movement (direction + magnitude) within semantic categories
align = displacement_stats["direction_alignment"]
mag_cons = displacement_stats["magnitude_consistency"]
n_cats = len(align)
mean_align = align.mean()
mean_mag = mag_cons.mean()

print("Consistency of movement within CDI semantic categories (movement = direction + magnitude)")
print("=" * 65)
print(f"Across {n_cats} semantic categories:")
print(f"  Mean direction alignment:   {mean_align:.3f}  (1 = same direction within category)")
print(f"  Mean magnitude consistency: {mean_mag:.3f}   (1 = similar distance within category)")
print()
print("Direction: High (≥0.7) / Moderate (0.4–0.7) / Low (<0.4):")
print(f"  {(align >= 0.7).sum()} / {((align >= 0.4) & (align < 0.7)).sum()} / {(align < 0.4).sum()} categories")
print("Magnitude: High (≥0.7) / Moderate (0.4–0.7) / Low (<0.4):")
print(f"  {(mag_cons >= 0.7).sum()} / {((mag_cons >= 0.4) & (mag_cons < 0.7)).sum()} / {(mag_cons < 0.4).sum()} categories")
print()
print("High consistency on BOTH direction and magnitude (≥0.6):")
both = displacement_stats[(align >= 0.6) & (mag_cons >= 0.6)][["direction_alignment", "magnitude_consistency"]]
print(both.to_string() if len(both) > 0 else "  (none)")
print()
print("Direction alignment by category (high = consistent direction):")
print(align.sort_values(ascending=False).to_string())
print()
print("Magnitude consistency by category (high = similar distances within category):")
print(mag_cons.sort_values(ascending=False).to_string())

## 7. Run for multiple result sets (optional)

You can run the same analysis for different embedding types (CLIP vs DINOv3) or different 2D methods (UMAP vs t-SNE) by loading their coordinate CSVs. Below we define a helper that builds the displacement + alignment stats for any CSV, then loop over available result directories.

In [19]:
def analyze_bv_things_csv(csv_path):
    """Load CSV, compute displacement and direction alignment by cdi_category. Returns (df, displacement_stats)."""
    df, bv_x, bv_y, things_x, things_y = load_bv_things_coordinates(csv_path)
    dx = (df[things_x] - df[bv_x]).values
    dy = (df[things_y] - df[bv_y]).values
    magnitude = np.sqrt(dx * dx + dy * dy)
    df = df.copy()
    df["displacement"] = magnitude
    eps = 1e-12
    length = np.maximum(magnitude, eps)
    df["dir_x"] = dx / length
    df["dir_y"] = dy / length
    grp = df.groupby("cdi_category", sort=True)
    disp = grp["displacement"].agg(["mean", "std", "count", "median"]).round(4)
    disp.columns = ["displacement_mean", "displacement_std", "n_words", "displacement_median"]
    alignment = grp.apply(lambda g: direction_alignment(g["dir_x"].values, g["dir_y"].values), include_groups=False)
    disp["direction_alignment"] = alignment.round(4)
    disp["displacement_cv"] = (disp["displacement_std"] / disp["displacement_mean"]).round(4)
    disp["magnitude_consistency"] = (1 / (1 + disp["displacement_cv"])).round(4)
    return df, disp.sort_values("displacement_mean", ascending=False)

# Example: run for CLIP UMAP and CLIP t-SNE (and DINOv3 if present)
for emb in ["clip", "dinov3"]:
    for method in ["umap", "tsne"]:
        folder = RESULTS_BASE / f"bv_things_results_{emb}"
        csv_name = f"{method}_bv_things_coordinates.csv"
        path = folder / csv_name
        if path.exists():
            _, stats = analyze_bv_things_csv(path)
            print(f"--- {emb.upper()} {method.upper()} ---")
            display(stats)

## 8. Save summary table (optional)

Save the per–semantic-category stats to a CSV for use in the manuscript or elsewhere. Re-run the cell above to set `displacement_stats` for the run you want to export (e.g. CLIP UMAP from section 4).

## Interpretation of results (how to read the numbers)

**What the metrics mean**

| Metric | Range | How to read it |
|--------|--------|-----------------|
| **displacement_mean** | 0 and up (in embedding axis units) | Average length of BV→THINGs arrows in that semantic category. *Larger* = baby and adult (THINGs) representations sit farther apart in the 2D space for that category. |
| **direction_alignment** | 0–1 | How similar the *direction* of arrows is within the category. *Close to 1* = arrows point the same way (consistent direction); *close to 0* = directions are scattered. |
| **magnitude_consistency** | 0–1 | How similar the *lengths* of arrows are within the category (1/(1+CV)). *Close to 1* = similar distances; *lower* = more variation in how far each word moves. |
| **displacement_cv** | 0 and up | Coefficient of variation (std/mean) of displacement. *Lower* = more consistent magnitude within the category. |

**What to conclude**

- **High direction alignment (e.g. > 0.95) across categories** → Within each semantic group, BV and THINGs differ in a *consistent direction*: e.g. all "animals" move from one region toward another in a similar way. So the *kind* of baby–adult difference is structured by semantic category.
- **High magnitude consistency (e.g. > 0.9)** → Within that category, words move similar *distances*; low magnitude consistency (e.g. "outside") means some words shift a lot and others less.
- **Large displacement_mean for a category** → On average, baby and adult representations for that category are farther apart in the 2D space (e.g. food_drink vs animals). **Small displacement_mean** → BV and THINGs are closer on average for that category (e.g. animals, body_parts).
- If both direction alignment and magnitude consistency are high for a category, **movement is consistent** in both direction and distance within that semantic group.

In [ ]:
# Optional: print a short interpretation from the stats table you have in memory (from section 4)
# If you ran section 4 on CLIP UMAP, this summarizes those results.
if "displacement_stats" in dir() and displacement_stats is not None:
    da = displacement_stats["direction_alignment"]
    mc = displacement_stats["magnitude_consistency"]
    dm = displacement_stats["displacement_mean"]
    print("Short interpretation (from current displacement_stats):")
    print(f"  Direction: mean alignment {da.mean():.3f}; all categories ≥0.97: {(da >= 0.97).all()}")
    print(f"  Magnitude: mean consistency {mc.mean():.3f}; lowest in: {mc.idxmin()} ({mc.min():.3f})")
    print(f"  Largest mean displacement: {dm.idxmax()} ({dm.max():.1f}); smallest: {dm.idxmin()} ({dm.min():.1f})")
else:
    print("Run section 4 first to populate displacement_stats.")

In [20]:
# Save the displacement_stats computed in section 4 (for the CSV you loaded there)
out_dir = RESULTS_BASE / "bv_things_results_clip"
out_dir.mkdir(parents=True, exist_ok=True)
out_path = out_dir / "bv_things_semantic_category_stats_umap.csv"
displacement_stats.to_csv(out_path)
print("Saved:", out_path)

## Summary and interpretation

**Movement is a vector** (direction + magnitude). We quantify both:

- **Magnitude (distance):** Mean displacement by semantic category tells you *how much* BV and THINGs differ on average. **Magnitude consistency** (1/(1+CV)) within each category tells you whether words in that category move similar distances—high = similar lengths, low = mixed.
- **Direction:** **Direction alignment** (mean resultant length) within each category tells you whether the BV→THINGs arrows point in a similar direction. High alignment = "BV and THINGs differ in similar ways" within that category (e.g. all animals shift from one region toward another in the same way).

If both direction alignment and magnitude consistency are high across categories, movements are consistent within each semantic category in both direction and distance.